In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import mysql.connector


def routes_from_page(driver):
    route_names = []
    route_links = []
    routes = driver.find_elements(By.XPATH, "//a[@class='route']")
    for route in routes:
        route_names.append(route.get_attribute('title'))
        route_links.append(route.get_attribute('href'))
    return route_names, route_links


# Function to extract bus details
def bus_details_extract(driver, route_name, route_link):
    bus_details_list = []
    time.sleep(3)  # Consider replacing this with a wait for specific elements

    bus_items = driver.find_elements(By.XPATH, '//div[contains(@class, "bus-item")]')

    for bus_item in bus_items:
        bus_details ={
            "Bus Name": bus_item.find_element(By.XPATH, './/div[contains(@class, "travels")]').text.strip() or 'N/A',

            "Bus Type": bus_item.find_element(By.XPATH, './/div[contains(@class, "bus-type")]').text.strip() or 'N/A',

            "Start of Journey": bus_item.find_element(By.XPATH,'.//div[contains(@class, "dp-time")]').text.strip() or 'N/A',

            "End of Journey": bus_item.find_element(By.XPATH,'.//div[contains(@class, "bp-time")]').text.strip() or 'N/A',

            "Duration": bus_item.find_element(By.XPATH, './/div[contains(@class, "dur")]').text.strip() or 'N/A',

            "Price": bus_item.find_element(By.XPATH,'.//div[contains(@class, "fare")]//span[contains(@class, "f-19 f-bold")]').text.strip() or 'N/A',

            "Star Rating": bus_item.find_element(By.XPATH,'.//div[contains(@class, "rating")]//span').text.strip() or 'N/A',

            "Seat Availability": bus_item.find_element(By.XPATH,'.//div[contains(@class, "seat-left")]').text.strip() or 'N/A',

            "Route Name": route_name,

            "Route Link": route_link
        }

        bus_details_list.append(bus_details)

    return bus_details_list

driver = webdriver.Chrome()

try:
    # Open the desired URL
    driver.get("https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile")
    wait = WebDriverWait(driver, 20)

    # Wait for the pagination container element
    pagination_container = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'DC_117_paginationTable')))

    all_route_names = []
    all_route_links = []

    # Iterate through each page
    for page in range(1, 3):  # Adjust the range based on the requirement
        xpath_expression = f'//div[contains(@class, "DC_117_pageTabs") and contains(text(), "{page}")]'
        page_button = pagination_container.find_element(By.XPATH, xpath_expression)

        actions = ActionChains(driver)
        actions.move_to_element(page_button).perform()
        page_button.click()
        time.sleep(2)

        route_names, route_links = routes_from_page(driver)
        all_route_names.extend(route_names)
        all_route_links.extend(route_links)

    all_bus_details = []

    for route_link, route_name in zip(all_route_links, all_route_names):
        driver.get(route_link)
        driver.maximize_window()
        time.sleep(2)

        try:
            view_buses_buttons = wait.until(
                EC.presence_of_all_elements_located(
                    (By.XPATH, "//div[@class='button' and contains(text(),'View Buses')]"))
            )
            time.sleep(5)

            for button in reversed(view_buses_buttons):
                try:
                    driver.execute_script("arguments[0].scrollIntoView(true);", button)
                    time.sleep(1)
                    button.click()
                    time.sleep(2)
                except Exception as e:
                    print(f"Error clicking button: {e}")
                    continue
        except Exception as e:
            print(f"Error during 'View Buses' button processing: {e}")

        # Scroll to the bottom of the page to ensure all buses are loaded
        scroll_pause_time = 2
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)

            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Extract bus details after all content is loaded
        try:
            bus_details = bus_details_extract(driver, route_name, route_link) #calling the function
            all_bus_details.extend(bus_details)
        except Exception as e:
            print(f"Error extracting bus details for route {route_name}: {e}")

finally:
    # Close the browser
    driver.quit()

# Convert bus details to a DataFrame and remove duplicates
df = pd.DataFrame(all_bus_details)
# Display the DataFrame
print(df.head())

user = 'root'
password = '2312'
host = '127.0.0.1'
database = 'redbus'

# Connect to the MySQL database
connection = mysql.connector.connect(user=user, password=password, host=host, database=database)
cursor = connection.cursor()

# Create the table if it doesn't exist
create_table_query = """
CREATE TABLE IF NOT EXISTS AndhraPradesh_Bus_Details(
    id INT AUTO_INCREMENT PRIMARY KEY,
    Bus_Name VARCHAR(100),
    Bus_Type VARCHAR(100),
    Start_of_Journey VARCHAR(100),
    End_of_Journey VARCHAR(100),
    Duration VARCHAR(100),
    Price FLOAT,
    Star_Rating FLOAT,
    Seat_Availability VARCHAR(100),
    Route_Name VARCHAR(255),
    Route_link VARCHAR(255)
)
"""
cursor.execute(create_table_query)

# Insert data into the table
for index, row in df.iterrows():
    try:
        insert_query = """
            INSERT INTO AndhraPradesh_Bus_Details (
                Bus_Name, Bus_Type, Start_of_Journey, End_of_Journey, Duration, Price, Star_Rating, Seat_Availability, Route_Name, Route_link
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        price = float(row["Price"].replace(",", "")) if row["Price"] not in ['N/A', ''] else None
        star_rating = float(row["Star Rating"]) if row["Star Rating"] not in ['N/A', ''] else None

        cursor.execute(insert_query, (
            row["Bus Name"], row["Bus Type"], row["Start of Journey"], row["End of Journey"], row["Duration"],
            price, star_rating,
            row["Seat Availability"], row["Route Name"], row["Route Link"]
        ))
    except Exception as e:
        print(f"Error inserting row {index}: {e}")

# Commit and close the connection
connection.commit()
cursor.close()
connection.close()

Error clicking button: Message: element click intercepted: Element <div class="button">...</div> is not clickable at point (1434, 15). Other element would receive the click: <div class="modify-sec-onward clearfix modify-sec d-color f-bold">...</div>
  (Session info: chrome=130.0.6723.117)
Stacktrace:
	GetHandleVerifier [0x00007FF7855338A5+3004357]
	(No symbol) [0x00007FF7851C9970]
	(No symbol) [0x00007FF78507582A]
	(No symbol) [0x00007FF7850CD80E]
	(No symbol) [0x00007FF7850CB2AC]
	(No symbol) [0x00007FF7850C8778]
	(No symbol) [0x00007FF7850C798C]
	(No symbol) [0x00007FF7850B996E]
	(No symbol) [0x00007FF7850EBBDA]
	(No symbol) [0x00007FF7850B92A6]
	(No symbol) [0x00007FF7850EBDF0]
	(No symbol) [0x00007FF78510BA4C]
	(No symbol) [0x00007FF7850EB983]
	(No symbol) [0x00007FF7850B7628]
	(No symbol) [0x00007FF7850B8791]
	GetHandleVerifier [0x00007FF78555A00D+3161901]
	GetHandleVerifier [0x00007FF7855AE060+3506048]
	GetHandleVerifier [0x00007FF7855A400D+3465005]
	GetHandleVerifier [0x00007FF7

Note: you may need to restart the kernel to use updated packages.
